### DK Salaries Optimizer

In [652]:
import pandas as pd
import numpy as np
from pulp import *

In [653]:
SALARY_CAP = 50000
MAX_PLAYERS = 8

In [654]:
dk_data = pd.read_csv("numberfire.csv")
dk_data.head()

,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,REB,AST,STL,BLK,TO,PLAYER_NAME
0,1,60.4,"$10,100",5.98,34.33,28.4,8.7,10.3,1.4,0.9,4.7,LeBron James PG/SF
1,3,54.7,"$10,000",5.47,34.63,27.0,9.6,9.4,1.1,0.4,3.8,Luka Doncic PG
2,5,51.8,"$10,600",4.88,35.32,21.6,11.7,7.5,1.3,0.8,3.6,Nikola Jokic C
3,7,50.6,"$10,200",4.96,32.67,21.8,9.4,9.8,1.6,0.4,3.3,Russell Westbrook PG
4,9,50.5,"$9,600",5.26,35.19,26.8,8.5,5.5,2.1,0.6,3.0,Kawhi Leonard SF


In [655]:
# drop rows that have nan
dk_data = dk_data.dropna(axis=0, how='any')
dk_data.head()

,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,REB,AST,STL,BLK,TO,PLAYER_NAME
0,1,60.4,"$10,100",5.98,34.33,28.4,8.7,10.3,1.4,0.9,4.7,LeBron James PG/SF
1,3,54.7,"$10,000",5.47,34.63,27.0,9.6,9.4,1.1,0.4,3.8,Luka Doncic PG
2,5,51.8,"$10,600",4.88,35.32,21.6,11.7,7.5,1.3,0.8,3.6,Nikola Jokic C
3,7,50.6,"$10,200",4.96,32.67,21.8,9.4,9.8,1.6,0.4,3.3,Russell Westbrook PG
4,9,50.5,"$9,600",5.26,35.19,26.8,8.5,5.5,2.1,0.6,3.0,Kawhi Leonard SF


In [656]:
def get_pos(name):
    l = name.split(' ')
    return l[-1] + '/UTIL'

# 'Giannis Antetokounmpo PF'.split('\xa0')

In [657]:
def get_first(name):
    l = name.split(' ')
    return l[0]

def get_last(name):
    l = name.split(' ')
    return l[1]

In [658]:
dk_data['PLAYER_FIRST'] = dk_data['PLAYER_NAME']
dk_data['PLAYER_LAST'] = dk_data['PLAYER_NAME']

In [659]:
# Get the fields that are useful
#dk_data = dk_data[['Name', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame']]
dk_data['Roster Position'] = dk_data.apply(lambda x: get_pos(x['PLAYER_NAME']),axis=1)
dk_data['PLAYER_FIRST'] = dk_data.apply(lambda x: get_first(x['PLAYER_FIRST']),axis=1)
dk_data['PLAYER_LAST'] = dk_data.apply(lambda x: get_last(x['PLAYER_LAST']),axis=1)


# change $s into int
dk_data['SALARY'] = dk_data['SALARY'].replace('[\$,]', '', regex=True).astype(float)

dk_data.head(10)

,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,REB,AST,STL,BLK,TO,PLAYER_NAME,PLAYER_FIRST,PLAYER_LAST,Roster Position
0,1,60.4,10100.0,5.98,34.33,28.4,8.7,10.3,1.4,0.9,4.7,LeBron James PG/SF,LeBron,James,PG/SF/UTIL
1,3,54.7,10000.0,5.47,34.63,27.0,9.6,9.4,1.1,0.4,3.8,Luka Doncic PG,Luka,Doncic,PG/UTIL
2,5,51.8,10600.0,4.88,35.32,21.6,11.7,7.5,1.3,0.8,3.6,Nikola Jokic C,Nikola,Jokic,C/UTIL
3,7,50.6,10200.0,4.96,32.67,21.8,9.4,9.8,1.6,0.4,3.3,Russell Westbrook PG,Russell,Westbrook,PG/UTIL
4,9,50.5,9600.0,5.26,35.19,26.8,8.5,5.5,2.1,0.6,3.0,Kawhi Leonard SF,Kawhi,Leonard,SF/UTIL
5,11,48.3,9400.0,5.14,35.39,28.8,3.9,7.7,1.4,0.3,3.0,Stephen Curry PG,Stephen,Curry,PG/UTIL
6,13,47.9,9700.0,4.94,34.08,25.3,9.6,3.2,1.2,1.9,2.5,Anthony Davis PF/C,Anthony,Davis,PF/C/UTIL
7,15,46.1,9300.0,4.95,34.17,25.7,7.0,5.2,1.1,0.4,2.2,Kevin Durant PF,Kevin,Durant,PF/UTIL
8,17,45.1,8700.0,5.18,34.02,23.7,6.5,5.3,1.9,0.4,3.4,Paul George SG,Paul,George,SG/UTIL
9,19,45.0,9000.0,5.00,33.97,26.7,4.7,5.6,1.3,0.2,2.4,Kyrie Irving PG,Kyrie,Irving,PG/UTIL


In [660]:
del dk_data['PLAYER_NAME']
dk_data['PLAYER_NAME'] = dk_data['PLAYER_FIRST'] + ' ' + dk_data['PLAYER_LAST']
dk_data

,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,REB,AST,STL,BLK,TO,PLAYER_FIRST,PLAYER_LAST,Roster Position,PLAYER_NAME
0,1,60.4,10100.0,5.98,34.33,28.4,8.7,10.3,1.4,0.9,4.7,LeBron,James,PG/SF/UTIL,LeBron James
1,3,54.7,10000.0,5.47,34.63,27.0,9.6,9.4,1.1,0.4,3.8,Luka,Doncic,PG/UTIL,Luka Doncic
2,5,51.8,10600.0,4.88,35.32,21.6,11.7,7.5,1.3,0.8,3.6,Nikola,Jokic,C/UTIL,Nikola Jokic
3,7,50.6,10200.0,4.96,32.67,21.8,9.4,9.8,1.6,0.4,3.3,Russell,Westbrook,PG/UTIL,Russell Westbrook
4,9,50.5,9600.0,5.26,35.19,26.8,8.5,5.5,2.1,0.6,3.0,Kawhi,Leonard,SF/UTIL,Kawhi Leonard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,387,0.0,5500.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,Marcus,Morris,SF/PF/UTIL,Marcus Morris
190,389,0.0,3000.0,0.00,0.11,0.0,0.0,0.0,0.0,0.0,0.0,Damian,Jones,C/UTIL,Damian Jones
191,391,0.0,3000.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,Abdel,Nader,SF/UTIL,Abdel Nader
192,393,0.0,3000.0,0.00,0.12,0.0,0.0,0.0,0.0,0.0,0.0,Cristiano,Felicio,C/UTIL,Cristiano Felicio


In [661]:
injured_players = ['LeBron James', 'Luka Doncic', 'test']
injured_players

['LeBron James', 'Luka Doncic', 'test']

In [662]:
dk_data[dk_data['PLAYER_NAME'].isin(injured_players)]

,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,REB,AST,STL,BLK,TO,PLAYER_FIRST,PLAYER_LAST,Roster Position,PLAYER_NAME
0,1,60.4,10100.0,5.98,34.33,28.4,8.7,10.3,1.4,0.9,4.7,LeBron,James,PG/SF/UTIL,LeBron James
1,3,54.7,10000.0,5.47,34.63,27.0,9.6,9.4,1.1,0.4,3.8,Luka,Doncic,PG/UTIL,Luka Doncic


In [663]:
dk_data = dk_data[~dk_data['PLAYER_NAME'].isin(injured_players)]

In [664]:
# Roster Positions for NBA 
nba_roster_positions = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UTIL']

SALARY_CAP = SALARY_CAP
MAX_PLAYERS = MAX_PLAYERS


def make_vars(name, position, roster_position):
    if position in roster_position:
        return LpVariable(name + '_' + position,lowBound=0, cat='Binary')
    return 0

for position in nba_roster_positions:
    dk_data[position] = dk_data.apply(lambda x: make_vars(x['PLAYER_NAME'],position, x['Roster Position']),axis=1)

    #[1 if position in x else 0 for x in dk_data['Roster Position']]

In [665]:
prob = LpProblem("OptimizeDK",LpMaximize)

In [666]:
names = dk_data['PLAYER_NAME']
salaries = dk_data['SALARY']
projected_points = dk_data['FP']

In [667]:
dk_data_check = dk_data

In [668]:
dk_data.head(30)

,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,REB,AST,STL,BLK,...,Roster Position,PLAYER_NAME,PG,SG,SF,PF,C,G,F,UTIL
2,5,51.8,10600.0,4.88,35.32,21.6,11.7,7.5,1.3,0.8,...,C/UTIL,Nikola Jokic,0,0,0,0,Nikola_Jokic_C,0,0,Nikola_Jokic_UTIL
3,7,50.6,10200.0,4.96,32.67,21.8,9.4,9.8,1.6,0.4,...,PG/UTIL,Russell Westbrook,Russell_Westbrook_PG,0,0,0,0,Russell_Westbrook_G,0,Russell_Westbrook_UTIL
4,9,50.5,9600.0,5.26,35.19,26.8,8.5,5.5,2.1,0.6,...,SF/UTIL,Kawhi Leonard,0,0,Kawhi_Leonard_SF,0,0,0,Kawhi_Leonard_F,Kawhi_Leonard_UTIL
5,11,48.3,9400.0,5.14,35.39,28.8,3.9,7.7,1.4,0.3,...,PG/UTIL,Stephen Curry,Stephen_Curry_PG,0,0,0,0,Stephen_Curry_G,0,Stephen_Curry_UTIL
6,13,47.9,9700.0,4.94,34.08,25.3,9.6,3.2,1.2,1.9,...,PF/C/UTIL,Anthony Davis,0,0,0,Anthony_Davis_PF,Anthony_Davis_C,0,Anthony_Davis_F,Anthony_Davis_UTIL
7,15,46.1,9300.0,4.95,34.17,25.7,7.0,5.2,1.1,0.4,...,PF/UTIL,Kevin Durant,0,0,0,Kevin_Durant_PF,0,0,Kevin_Durant_F,Kevin_Durant_UTIL
8,17,45.1,8700.0,5.18,34.02,23.7,6.5,5.3,1.9,0.4,...,SG/UTIL,Paul George,0,Paul_George_SG,0,0,0,Paul_George_G,0,Paul_George_UTIL
9,19,45.0,9000.0,5.00,33.97,26.7,4.7,5.6,1.3,0.2,...,PG/UTIL,Kyrie Irving,Kyrie_Irving_PG,0,0,0,0,Kyrie_Irving_G,0,Kyrie_Irving_UTIL
10,21,44.5,9800.0,4.54,35.71,27.0,4.6,6.8,0.8,0.3,...,PG/UTIL,Damian Lillard,Damian_Lillard_PG,0,0,0,0,Damian_Lillard_G,0,Damian_Lillard_UTIL
11,23,43.5,7100.0,6.12,34.82,27.9,4.5,4.4,1.1,0.2,...,PG/SG/UTIL,Donovan Mitchell,Donovan_Mitchell_PG,Donovan_Mitchell_SG,0,0,0,Donovan_Mitchell_G,0,Donovan_Mitchell_UTIL


In [669]:
# The objective function is added to 'prob' first
# OBJ FUNCTION
prob += lpSum([dk_data[i] * projected_points for i in nba_roster_positions]), 'Projected Points'

In [670]:
# CONSTRAINTS

# Stay under salary cap
prob += lpSum(salaries * dk_data[i] for i in nba_roster_positions) <= SALARY_CAP

# Stay under Num Players
prob += lpSum(dk_data[i] for i in nba_roster_positions) <= MAX_PLAYERS

# Select one of each position
for position in nba_roster_positions:
    prob += lpSum([dk_data[position]]) >= 1

# Don't use same player twice
for index, row in dk_data.iterrows():
    prob += lpSum(row[i] for i in nba_roster_positions) <= 1


In [671]:
# prob

In [672]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [673]:
prob.status

1

In [698]:
print("Total Estimated Points = ", value(prob.objective))
print(value(prob.objective))

Total Estimated Points =  296.8
296.8


In [718]:
df_final = pd.DataFrame(columns=["names"])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue == 1:
        print(v.name)
        v.getName
        df_final = df_final.append({"names":v.name}, ignore_index=True)

Carmelo_Anthony_PF
Chris_Paul_PG
Donovan_Mitchell_SG
Gorgui_Dieng_C
Kawhi_Leonard_UTIL
Stephen_Curry_G
Timothe_Luwawu_Cabarrot_SF
Will_Barton_F


In [719]:
df_final

,names
0,Carmelo_Anthony_PF
1,Chris_Paul_PG
2,Donovan_Mitchell_SG
3,Gorgui_Dieng_C
4,Kawhi_Leonard_UTIL
5,Stephen_Curry_G
6,Timothe_Luwawu_Cabarrot_SF
7,Will_Barton_F


In [727]:
# df_final = df_final.names.str.split("_", expand = True)
# df_final.columns = ['first','PLAYER_NAME','position','f']
# df_final['PLAYER_NAME'] = df_final['first'] + ' ' + df_final['PLAYER_NAME']

import time
import datetime
# filename = datetime.date.today()
mydate = datetime.datetime.now()
csvstr = datetime.datetime.strftime(mydate, '%Y, %m, %d')
df_final.to_csv(csvstr + ' lineup.csv', sep = ',')
df_final

,first,PLAYER_NAME,position,f
0,Carmelo,Carmelo Anthony,PF,None
1,Chris,Chris Paul,PG,None
2,Donovan,Donovan Mitchell,SG,None
3,Gorgui,Gorgui Dieng,C,None
4,Kawhi,Kawhi Leonard,UTIL,None
5,Stephen,Stephen Curry,G,None
6,Timothe,Timothe Luwawu,Cabarrot,SF
7,Will,Will Barton,F,None


In [729]:
check_salaries = df_final.merge(dk_data_check, on = 'PLAYER_NAME', how = 'left')
check_salaries

,first,PLAYER_NAME,position,f,Unnamed: 0,FP,SALARY,VALUE,MIN,PTS,...,PG,SG,SF,PF,C,G,F,UTIL,merge_column_lower,Test3
0,Carmelo,Carmelo Anthony,PF,None,97.0,26.5,4300.0,6.16,24.05,14.3,...,0,0,0,Carmelo_Anthony_PF,0,0,Carmelo_Anthony_F,Carmelo_Anthony_UTIL,carmelo anthony,anthony
1,Chris,Chris Paul,PG,None,37.0,37.3,7000.0,5.33,32.59,16.2,...,Chris_Paul_PG,0,0,0,0,Chris_Paul_G,0,Chris_Paul_UTIL,chris paul,paul
2,Donovan,Donovan Mitchell,SG,None,23.0,43.5,7100.0,6.12,34.82,27.9,...,Donovan_Mitchell_PG,Donovan_Mitchell_SG,0,0,0,Donovan_Mitchell_G,0,Donovan_Mitchell_UTIL,donovan mitchell,mitchell
3,Gorgui,Gorgui Dieng,C,None,65.0,30.8,3900.0,7.90,19.78,13.3,...,0,0,0,0,Gorgui_Dieng_C,0,0,Gorgui_Dieng_UTIL,gorgui dieng,dieng
4,Kawhi,Kawhi Leonard,UTIL,None,9.0,50.5,9600.0,5.26,35.19,26.8,...,0,0,Kawhi_Leonard_SF,0,0,0,Kawhi_Leonard_F,Kawhi_Leonard_UTIL,kawhi leonard,leonard
5,Stephen,Stephen Curry,G,None,11.0,48.3,9400.0,5.14,35.39,28.8,...,Stephen_Curry_PG,0,0,0,0,Stephen_Curry_G,0,Stephen_Curry_UTIL,stephen curry,curry
6,Timothe,Timothe Luwawu,Cabarrot,SF,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Will,Will Barton,F,None,47.0,34.7,5400.0,6.43,33.34,14.7,...,0,Will_Barton_SG,Will_Barton_SF,0,0,Will_Barton_G,Will_Barton_F,Will_Barton_UTIL,will barton,barton


In [733]:
SALARY_CHECK = check_salaries[['PLAYER_NAME','SALARY']]
SALARY_CHECK

,PLAYER_NAME,SALARY
0,Carmelo Anthony,4300.0
1,Chris Paul,7000.0
2,Donovan Mitchell,7100.0
3,Gorgui Dieng,3900.0
4,Kawhi Leonard,9600.0
5,Stephen Curry,9400.0
6,Timothe Luwawu,NaN
7,Will Barton,5400.0


In [ ]:
# def str_merge(part_string_df,full_string_df, merge_column):
#     merge_column_lower = 'merge_column_lower'
#     part_string_df[merge_column_lower] = part_string_df[merge_column].str.lower()
#     full_string_df[merge_column_lower] = full_string_df[merge_column].str.lower()   
#     pat = '|'.join(r"{}".format(x) for x in part_string_df[merge_column_lower])
#     full_string_df['Test3'] = full_string_df[merge_column_lower].str.extract('('+ pat + ')', expand=True)
#     DF = pd.merge(part_string_df, full_string_df, left_on= merge_column_lower, right_on='Test3').drop([merge_column_lower + '_x',merge_column_lower + '_y','Test3'],axis=1)
#     return DF

# check_salaries_full = str_merge(
#     part_string_df = df_final,
#     full_string_df = dk_data_check, 
#     merge_column = 'PLAYER_NAME')